# Scrapping dolar y mandarlo a Excel

## Get dolars

In [1]:
import os
import pandas as pd
from bs4 import BeautifulSoup
import requests

In [2]:
urls =  {"dolar_oficial": "https://dolarhoy.com/cotizaciondolaroficial",
         "dolar_blue": "https://dolarhoy.com/cotizaciondolarblue"}

def get_dolars(urls: dict[str]) -> tuple:
    
    try:
        # get html 
        response_ofi = requests.get(urls["dolar_oficial"])
        response_ofi.raise_for_status()
        response_blue = requests.get(urls["dolar_blue"])
        response_blue.raise_for_status()
        
        # parse html to soup object
        soup_ofi = BeautifulSoup(response_ofi.text, 'html.parser')
        soup_blue = BeautifulSoup(response_blue.text, 'html.parser')
        
        # find particular tags and attributes, .text method to get only the text inside the div
        dolar_ofi = soup_ofi.find("div", class_="value").text
        dolar_blue = soup_blue.find("div", class_="value").text

        # Sacamos el '$' y convertimos a float
        dolar_ofi = float(dolar_ofi.lstrip("$"))
        dolar_blue = float(dolar_blue.lstrip("$"))
        # print(dolar_ofi)
        # print(dolar_blue)
        
    except Exception as e:
        print(e)

    return dolar_ofi, dolar_blue
    

In [3]:
# get data
dolar_oficial, dolar_blue = get_dolars(urls)

In [4]:
print(f"Dolar Oficial: {dolar_oficial}\nDolar Blue: {dolar_blue}")

Dolar Oficial: 894.0
Dolar Blue: 1035.0


## Adding data to Excel sheet

In [5]:
data = "Mis compras.xlsx"
df = pd.read_excel(data, sheet_name=0, index_col=0)

In [6]:
df.head()

,Precio inical,ahora,aumento,% Aumento,Hoy
Item,,,,,
Audiotechnica m30x,18000,143000,125000,694.44%,2024-04-26
Presonus AudioboxGo,16000,149000,133000,831.25%,2024-04-26
Behringer xm8500,11000,55000,44000,400%,2024-04-26
Silla Ergonomica Femnto,17000,199000,182000,1070.59%,2024-04-26


In [13]:
df.drop("Hoy", axis=1, inplace=True)
df.head()

,Precio inical,ahora,aumento,% Aumento,Dolar oficial,Dolar blue
Item,,,,,,
Audiotechnica m30x,18000,143000,125000,694.44%,894.0,1035.0
Presonus AudioboxGo,16000,149000,133000,831.25%,894.0,1035.0
Behringer xm8500,11000,55000,44000,400%,894.0,1035.0
Silla Ergonomica Femnto,17000,199000,182000,1070.59%,894.0,1035.0


In [15]:
from datetime import date
today = date.today()
df["Hoy"] = today
df.head()

,Precio inical,ahora,aumento,% Aumento,Dolar oficial,Dolar blue,Hoy
Item,,,,,,,
Audiotechnica m30x,18000,143000,125000,694.44%,894.0,1035.0,2024-04-26
Presonus AudioboxGo,16000,149000,133000,831.25%,894.0,1035.0,2024-04-26
Behringer xm8500,11000,55000,44000,400%,894.0,1035.0,2024-04-26
Silla Ergonomica Femnto,17000,199000,182000,1070.59%,894.0,1035.0,2024-04-26


In [16]:
df["Hoy"] = pd.to_datetime(df["Hoy"], format='%d-%m-%Y', dayfirst=True)
df["Hoy"] = df["Hoy"].dt.strftime('%d-%m-%Y')
df.Hoy

Item
Audiotechnica m30x         26-04-2024
Presonus AudioboxGo        26-04-2024
Behringer xm8500           26-04-2024
Silla Ergonomica Femnto    26-04-2024
Name: Hoy, dtype: object

### Create columns and add the data

In [17]:
df["Dolar oficial"] = dolar_oficial
df["Dolar blue"] = dolar_blue
df.head()

,Precio inical,ahora,aumento,% Aumento,Dolar oficial,Dolar blue,Hoy
Item,,,,,,,
Audiotechnica m30x,18000,143000,125000,694.44%,894.0,1035.0,26-04-2024
Presonus AudioboxGo,16000,149000,133000,831.25%,894.0,1035.0,26-04-2024
Behringer xm8500,11000,55000,44000,400%,894.0,1035.0,26-04-2024
Silla Ergonomica Femnto,17000,199000,182000,1070.59%,894.0,1035.0,26-04-2024


In [19]:
with pd.ExcelWriter("output.xlsx") as writer:
    df.to_excel(writer)